In [1]:
import numpy as np
import pandas as pd
import pickle
import xgboost
import os
from scipy.stats import ttest_1samp, ttest_ind, ttest_rel
from config import path_prefix_results
import warnings

warnings.filterwarnings('ignore')


## Create latex template for leg, loin, belly, groin, shoulder, body results

In [2]:
#latex output setup
latex_template = r'''\documentclass[a4paper]{{article}}
\usepackage{{booktabs}}
\usepackage{{pdflscape}}
\begin{{document}}
\thispagestyle{{empty}}
\begin{{landscape}}
{}
\end{{landscape}}
\end{{document}}
'''

path_leg_r2_latex= os.path.join(path_prefix_results, 'leg_r2.tex')
path_leg_rmse_latex= os.path.join(path_prefix_results, 'leg_rmse.tex')
path_leg_ttest_latex= os.path.join(path_prefix_results, 'leg_ttest.tex')
path_leg_fs_ttest_diff_latex= os.path.join(path_prefix_results, 'leg_fs_ttest_diff.tex')
path_leg_fs_ttest_diff_cross_latex= os.path.join(path_prefix_results, 'leg_fs_ttest_diff_cross.tex')

path_loin_r2_latex= os.path.join(path_prefix_results, 'loin_r2.tex')
path_loin_rmse_latex= os.path.join(path_prefix_results, 'loin_rmse.tex')
path_loin_ttest_latex= os.path.join(path_prefix_results, 'loin_ttest.tex')
path_loin_fs_ttest_diff_latex= os.path.join(path_prefix_results, 'loin_fs_ttest_diff.tex')
path_loin_fs_ttest_diff_cross_latex= os.path.join(path_prefix_results, 'loin_fs_ttest_diff_cross.tex')

path_belly_r2_latex= os.path.join(path_prefix_results, 'belly_r2.tex')
path_belly_rmse_latex= os.path.join(path_prefix_results, 'belly_rmse.tex')
path_belly_ttest_latex= os.path.join(path_prefix_results, 'belly_ttest.tex')
path_belly_fs_ttest_diff_latex= os.path.join(path_prefix_results, 'belly_fs_ttest_diff.tex')
path_belly_fs_ttest_diff_cross_latex= os.path.join(path_prefix_results, 'belly_fs_ttest_diff_cross.tex')

path_groin_r2_latex= os.path.join(path_prefix_results, 'groin_r2.tex')
path_groin_rmse_latex= os.path.join(path_prefix_results, 'groin_rmse.tex')
path_groin_ttest_latex= os.path.join(path_prefix_results, 'groin_ttest.tex')
path_groin_fs_ttest_diff_latex= os.path.join(path_prefix_results, 'groin_fs_ttest_diff.tex')
path_groin_fs_ttest_diff_cross_latex= os.path.join(path_prefix_results, 'groin_fs_ttest_diff_cross.tex')

path_shoulder_r2_latex= os.path.join(path_prefix_results, 'shoulder_r2.tex')
path_shoulder_rmse_latex= os.path.join(path_prefix_results, 'shoulder_rmse.tex')
path_shoulder_ttest_latex= os.path.join(path_prefix_results, 'shoulder_ttest.tex')
path_shoulder_fs_ttest_diff_latex= os.path.join(path_prefix_results, 'shoulder_fs_ttest_diff.tex')
path_shoulder_fs_ttest_diff_cross_latex= os.path.join(path_prefix_results, 'shoulder_fs_ttest_diff_cross.tex')

path_body_r2_latex= os.path.join(path_prefix_results, 'body_r2.tex')
path_body_rmse_latex= os.path.join(path_prefix_results, 'body_rmse.tex')
path_body_ttest_latex= os.path.join(path_prefix_results, 'body_ttest.tex')
path_body_fs_ttest_diff_latex= os.path.join(path_prefix_results, 'body_fs_ttest_diff.tex')
path_body_fs_ttest_diff_cross_latex= os.path.join(path_prefix_results, 'body_fs_ttest_diff_cross.tex')

In [3]:
def test_diff_cross(df, row='multi-atlas'):
    tmp= pd.DataFrame(index=df.columns, columns=df.columns)
    for i in df.columns:
        for j in df.columns:
            tmp.loc[i,j]= ttest_rel(np.abs(df.loc[row, i]), np.abs(df.loc[row, j]))[1]
    return tmp

def test_diff(df, col='multi-atlas'):
    tmp= pd.DataFrame(index=df.index, columns=df.columns)
    for i in df.index:
        for j in df.columns:
            tmp.loc[i,j]= ttest_rel(np.abs(df.loc[i, j]), np.abs(df.loc[col, j]))[1]
    tmp= tmp.drop(col, axis='rows')
    return tmp

def test_compare(df0, df1):
    tmp= pd.DataFrame(index=df0.index, columns=df0.columns)
    for i in df0.index:
        for j in df0.columns:
            tmp.loc[i,j]= ttest_rel(np.abs(df0.loc[i, j]), np.abs(df1.loc[i, j]))[1]
    return tmp

## Load data 

In [4]:
data_fs= pickle.load(open(os.path.join(path_prefix_results,'results.pickle'), 'rb'))
data_no_fs= pickle.load(open(os.path.join(path_prefix_results,'results_no_fs.pickle'), 'rb'))

In [5]:
leg_fs= data_fs[data_fs['dataset'] == 'leg']
loin_fs= data_fs[data_fs['dataset'] == 'loin']
belly_fs= data_fs[data_fs['dataset'] == 'belly']
groin_fs= data_fs[data_fs['dataset'] == 'groin']
shoulder_fs= data_fs[data_fs['dataset'] == 'shoulder']
body_fs= data_fs[data_fs['dataset'] == 'body']

leg_no_fs= data_no_fs[data_no_fs['dataset'] == 'leg']
loin_no_fs= data_no_fs[data_no_fs['dataset'] == 'loin']
belly_no_fs= data_no_fs[data_no_fs['dataset'] == 'belly']
groin_no_fs= data_no_fs[data_no_fs['dataset'] == 'groin']
shoulder_no_fs= data_no_fs[data_no_fs['dataset'] == 'shoulder']
body_no_fs= data_no_fs[data_no_fs['dataset'] == 'body']

In [6]:
rename_dict_regr= {'KNNR_Objective': 'kNN', 'LinearRegression_Objective': 'linear', 'PLSRegression_Objective': 'PLS', 'RidgeRegression_Objective': 'ridge', 'LassoRegression_Objective': 'lasso'}
#rename_dict_type= {'203a': 'atlas 1', '203k': 'atlas 2', '204f': 'atlas 3', '206k': 'atlas 4', '208f': 'atlas 5', 'all': 'multi-atlas', 'mean_mask': 'mean'}
#row_order= ['atlas 1', 'atlas 2', 'atlas 3', 'atlas 4', 'atlas 5', 'mean', 'multi-atlas']
rename_dict_type= sorted(leg_fs['type'].unique())
rename_dict_type= dict(map(lambda x, i: (x, f"atlas {i+1}"), rename_dict_type,  range(len(rename_dict_type))))
rename_dict_type['all']= "multi-atlas"
rename_dict_type['mean_mask'] = "mean"
print(rename_dict_type)
row_order= list(rename_dict_type.values())
col_order= ['linear', 'PLS', 'lasso', 'ridge', 'kNN']

{'all': 'multi-atlas', 'mean_mask': 'mean', 'p004': 'atlas 3', 'p029': 'atlas 4', 'p032': 'atlas 5', 'p041': 'atlas 6', 'p048': 'atlas 7'}


## leg $r^2$ results

In [7]:
leg_fs_results= leg_fs.pivot(index='type', columns='model', values='r2_1')
leg_fs_results= leg_fs_results.rename(rename_dict_regr, axis='columns')
leg_fs_results= leg_fs_results.rename(rename_dict_type)
leg_fs_results= leg_fs_results.apply(lambda x: np.round(x, 4))
leg_fs_results= leg_fs_results.loc[row_order, col_order]

leg_no_fs_results= leg_no_fs.pivot(index='type', columns='model', values='r2_1')
leg_no_fs_results= leg_no_fs_results.rename(rename_dict_regr, axis='columns')
leg_no_fs_results= leg_no_fs_results.rename(rename_dict_type)
leg_no_fs_results= leg_no_fs_results.apply(lambda x: np.round(x, 4))
leg_no_fs_results= leg_no_fs_results.loc[row_order, col_order]

leg_all= pd.concat([leg_fs_results, leg_no_fs_results], axis='columns')
columns= pd.MultiIndex.from_product([['feature selection', 'no feature selection'], col_order])
leg_all.columns= columns

In [8]:
leg_all_r2_latex= leg_all.to_latex()
print(leg_all_r2_latex)

#save results to tex file
with open(path_leg_r2_latex, 'w') as f:
    f.write(latex_template.format(leg_all_r2_latex))

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} & \multicolumn{5}{l}{feature selection} & \multicolumn{5}{l}{no feature selection} \\
{} &            linear &     PLS &   lasso &   ridge &     kNN &               linear &     PLS &   lasso &   ridge &     kNN \\
type        &                   &         &         &         &         &                      &         &         &         &         \\
\midrule
multi-atlas &            0.9844 &  0.9842 &  0.9792 &  0.9896 &  0.9044 &               0.9244 &  0.9480 &  0.9542 &  0.9447 &  0.7617 \\
mean        &            0.9681 &  0.9657 &  0.9679 &  0.9715 &  0.8983 &               0.9346 &  0.9458 &  0.9557 &  0.9524 &  0.7521 \\
atlas 3     &            0.9719 &  0.9707 &  0.9740 &  0.9744 &  0.9084 &               0.9098 &  0.9490 &  0.9615 &  0.9606 &  0.7852 \\
atlas 4     &            0.9578 &  0.9627 &  0.9610 &  0.9623 &  0.8625 &               0.8221 &  0.9409 &  0.9420 &  0.9388 &  0.6730 \\
atlas 5     &            0.9647 &  0.9643 &  

## leg RMSE results

In [9]:
leg_fs_results_rmse= leg_fs.pivot(index='type', columns='model', values='rmse_1')
leg_fs_results_rmse= leg_fs_results_rmse.rename(rename_dict_regr, axis='columns')
leg_fs_results_rmse= leg_fs_results_rmse.rename(rename_dict_type)
leg_fs_results_rmse= leg_fs_results_rmse.apply(lambda x: np.round(x, 4))
leg_fs_results_rmse= leg_fs_results_rmse.loc[row_order, col_order]

leg_no_fs_results_rmse= leg_no_fs.pivot(index='type', columns='model', values='rmse_1')
leg_no_fs_results_rmse= leg_no_fs_results_rmse.rename(rename_dict_regr, axis='columns')
leg_no_fs_results_rmse= leg_no_fs_results_rmse.rename(rename_dict_type)
leg_no_fs_results_rmse= leg_no_fs_results_rmse.apply(lambda x: np.round(x, 4))
leg_no_fs_results_rmse= leg_no_fs_results_rmse.loc[row_order, col_order]

leg_all_rmse= pd.concat([leg_fs_results_rmse, leg_no_fs_results_rmse], axis='columns')
columns= pd.MultiIndex.from_product([['feature selection', 'no feature selection'], col_order])
leg_all_rmse.columns= columns

In [10]:
leg_all_rmse_latex= leg_all_rmse.to_latex()
print(leg_all_rmse_latex)

#save results to tex file
with open(path_leg_rmse_latex, 'w') as f:
    f.write(latex_template.format(leg_all_rmse_latex))

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} & \multicolumn{5}{l}{feature selection} & \multicolumn{5}{l}{no feature selection} \\
{} &            linear &       PLS &     lasso &     ridge &       kNN &               linear &       PLS &     lasso &     ridge &       kNN \\
type        &                   &           &           &           &           &                      &           &           &           &           \\
\midrule
multi-atlas &          139.6655 &  140.7403 &  161.6738 &  114.2854 &  346.1628 &             307.8653 &  255.2184 &  239.6201 &  263.2878 &  546.6112 \\
mean        &          199.8389 &  207.2478 &  200.5416 &  188.9756 &  357.1571 &             286.4517 &  260.7895 &  235.6104 &  244.3274 &  557.5321 \\
atlas 3     &          187.6540 &  191.7631 &  180.7159 &  179.3265 &  338.8735 &             336.2831 &  252.8250 &  219.6559 &  222.1540 &  518.9570 \\
atlas 4     &          229.9407 &  216.3742 &  221.1468 &  217.4927 &  415.1697 &             472.2728 

## t-test leg

In [11]:
leg_fs['diff']= (leg_fs['y_pred_1'] - leg_fs['y_test_1'])**2
leg_fs_test= leg_fs.pivot(index='type', columns='model', values='diff')
leg_fs_test= leg_fs_test.rename(rename_dict_regr, axis='columns')
leg_fs_test= leg_fs_test.rename(rename_dict_type)
leg_fs_test= leg_fs_test.loc[row_order, col_order]

leg_no_fs['diff']= (leg_no_fs['y_pred_1'] - leg_no_fs['y_test_1'])**2
leg_no_fs_test= leg_no_fs.pivot(index='type', columns='model', values='diff')
leg_no_fs_test= leg_no_fs_test.rename(rename_dict_regr, axis='columns')
leg_no_fs_test= leg_no_fs_test.rename(rename_dict_type)
leg_no_fs_test= leg_no_fs_test.loc[row_order, col_order]

In [12]:
leg_ttest_latex= test_compare(leg_fs_test, leg_no_fs_test).applymap(lambda x: '%.0e' % x).to_latex()
print(leg_ttest_latex)

#save results to tex file
with open(path_leg_ttest_latex, 'w') as f:
    f.write(latex_template.format(leg_ttest_latex))

\begin{tabular}{llllll}
\toprule
model & linear &    PLS &  lasso &  ridge &    kNN \\
type        &        &        &        &        &        \\
\midrule
multi-atlas &  2e-49 &  3e-68 &  5e-57 &  3e-70 &  2e-54 \\
mean        &  3e-30 &  1e-23 &  1e-12 &  1e-37 &  4e-54 \\
atlas 3     &  1e-48 &  6e-30 &  2e-26 &  4e-34 &  9e-45 \\
atlas 4     &  5e-49 &  3e-26 &  3e-21 &  2e-16 &  4e-29 \\
atlas 5     &  6e-57 &  2e-20 &  1e-22 &  1e-18 &  7e-41 \\
atlas 6     &  3e-36 &  1e-20 &  3e-02 &  1e-48 &  2e-46 \\
atlas 7     &  3e-61 &  5e-24 &  2e-10 &  6e-22 &  5e-79 \\
\bottomrule
\end{tabular}



In [13]:
leg_fs_ttest_diff_latex= test_diff(leg_fs_test).applymap(lambda x: '%.0e' % x).to_latex()
print(leg_fs_ttest_diff_latex)

#save results to tex file
with open(path_leg_fs_ttest_diff_latex, 'w') as f:
    f.write(latex_template.format(leg_fs_ttest_diff_latex))

\begin{tabular}{llllll}
\toprule
model & linear &    PLS &  lasso &  ridge &    kNN \\
type    &        &        &        &        &        \\
\midrule
mean    &  7e-23 &  2e-33 &  3e-18 &  2e-51 &  2e-01 \\
atlas 3 &  1e-16 &  5e-18 &  6e-05 &  6e-29 &  4e-01 \\
atlas 4 &  3e-36 &  2e-26 &  2e-22 &  8e-32 &  2e-10 \\
atlas 5 &  7e-36 &  2e-37 &  3e-21 &  8e-40 &  3e-01 \\
atlas 6 &  3e-42 &  1e-28 &  1e-27 &  8e-32 &  2e-11 \\
atlas 7 &  1e-20 &  5e-31 &  4e-24 &  7e-50 &  4e-01 \\
\bottomrule
\end{tabular}



In [14]:
leg_fs_ttest_diff_cross_latex= test_diff_cross(leg_fs_test.loc[['multi-atlas']]).applymap(lambda x: '%.0e' % x).to_latex()
print(leg_fs_ttest_diff_cross_latex)

#save results to tex file
with open(path_leg_fs_ttest_diff_cross_latex, 'w') as f:
    f.write(latex_template.format(leg_fs_ttest_diff_cross_latex))

\begin{tabular}{llllll}
\toprule
model & linear &    PLS &  lasso &  ridge &    kNN \\
model  &        &        &        &        &        \\
\midrule
linear &    nan &  8e-01 &  5e-08 &  2e-13 &  1e-27 \\
PLS    &  8e-01 &    nan &  4e-16 &  3e-18 &  7e-27 \\
lasso  &  5e-08 &  4e-16 &    nan &  1e-36 &  7e-24 \\
ridge  &  2e-13 &  3e-18 &  1e-36 &    nan &  3e-31 \\
kNN    &  1e-27 &  7e-27 &  7e-24 &  3e-31 &    nan \\
\bottomrule
\end{tabular}



## loin $r^2$ results

In [15]:
loin_fs_results= loin_fs.pivot(index='type', columns='model', values='r2_1')
loin_fs_results= loin_fs_results.rename(rename_dict_regr, axis='columns')
loin_fs_results= loin_fs_results.rename(rename_dict_type)
loin_fs_results= loin_fs_results.apply(lambda x: np.round(x, 4))
loin_fs_results= loin_fs_results.loc[row_order, col_order]

loin_no_fs_results= loin_no_fs.pivot(index='type', columns='model', values='r2_1')
loin_no_fs_results= loin_no_fs_results.rename(rename_dict_regr, axis='columns')
loin_no_fs_results= loin_no_fs_results.rename(rename_dict_type)
loin_no_fs_results= loin_no_fs_results.apply(lambda x: np.round(x, 4))
loin_no_fs_results= loin_no_fs_results.loc[row_order, col_order]

loin_all= pd.concat([loin_fs_results, loin_no_fs_results], axis='columns')
columns= pd.MultiIndex.from_product([['feature selection', 'no feature selection'], col_order])
loin_all.columns= columns

In [16]:
loin_all_r2_latex= loin_all.to_latex()
print(loin_all_r2_latex)

#save results to tex file
with open(path_loin_r2_latex, 'w') as f:
    f.write(latex_template.format(loin_all_r2_latex))

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} & \multicolumn{5}{l}{feature selection} & \multicolumn{5}{l}{no feature selection} \\
{} &            linear &     PLS &   lasso &   ridge &     kNN &               linear &     PLS &   lasso &   ridge &     kNN \\
type        &                   &         &         &         &         &                      &         &         &         &         \\
\midrule
multi-atlas &            0.9402 &  0.9617 &  0.9591 &  0.9668 &  0.8999 &               0.5925 &  0.8912 &  0.8686 &  0.8761 &  0.7460 \\
mean        &            0.9344 &  0.9323 &  0.9344 &  0.9344 &  0.8586 &               0.6938 &  0.8934 &  0.9067 &  0.8951 &  0.7263 \\
atlas 3     &            0.9261 &  0.9261 &  0.9260 &  0.9301 &  0.8345 &               0.6651 &  0.8896 &  0.8992 &  0.8931 &  0.6940 \\
atlas 4     &            0.9392 &  0.9382 &  0.9385 &  0.9363 &  0.8462 &               0.7472 &  0.9124 &  0.9186 &  0.9082 &  0.7015 \\
atlas 5     &            0.9246 &  0.9268 &  

## loin RMSE results

In [17]:
loin_fs_results_rmse= loin_fs.pivot(index='type', columns='model', values='rmse_1')
loin_fs_results_rmse= loin_fs_results_rmse.rename(rename_dict_regr, axis='columns')
loin_fs_results_rmse= loin_fs_results_rmse.rename(rename_dict_type)
loin_fs_results_rmse= loin_fs_results_rmse.apply(lambda x: np.round(x, 4))
loin_fs_results_rmse= loin_fs_results_rmse.loc[row_order, col_order]

loin_no_fs_results_rmse= loin_no_fs.pivot(index='type', columns='model', values='rmse_1')
loin_no_fs_results_rmse= loin_no_fs_results_rmse.rename(rename_dict_regr, axis='columns')
loin_no_fs_results_rmse= loin_no_fs_results_rmse.rename(rename_dict_type)
loin_no_fs_results_rmse= loin_no_fs_results_rmse.apply(lambda x: np.round(x, 4))
loin_no_fs_results_rmse= loin_no_fs_results_rmse.loc[row_order, col_order]

loin_all_rmse= pd.concat([loin_fs_results_rmse, loin_no_fs_results_rmse], axis='columns')
columns= pd.MultiIndex.from_product([['feature selection', 'no feature selection'], col_order])
loin_all_rmse.columns= columns

In [18]:
loin_all_rmse_latex= loin_all_rmse.to_latex()
print(loin_all_rmse_latex)

#save results to tex file
with open(path_loin_rmse_latex, 'w') as f:
    f.write(latex_template.format(loin_all_rmse_latex))

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} & \multicolumn{5}{l}{feature selection} & \multicolumn{5}{l}{no feature selection} \\
{} &            linear &       PLS &     lasso &     ridge &       kNN &               linear &       PLS &     lasso &     ridge &       kNN \\
type        &                   &           &           &           &           &                      &           &           &           &           \\
\midrule
multi-atlas &          243.6113 &  195.0351 &  201.4013 &  181.4516 &  315.2186 &             636.0598 &  328.5790 &  361.2460 &  350.7629 &  502.2013 \\
mean        &          255.1292 &  259.2060 &  255.1447 &  255.1288 &  374.6675 &             551.3306 &  325.3102 &  304.3034 &  322.6615 &  521.2670 \\
atlas 3     &          270.8934 &  270.8934 &  270.9708 &  263.5031 &  405.3406 &             576.5713 &  331.1058 &  316.3502 &  325.7495 &  551.1847 \\
atlas 4     &          245.7000 &  247.7660 &  247.1038 &  251.5631 &  390.8042 &             500.9483 

## t-test loin

In [19]:
loin_fs['diff']= (loin_fs['y_pred_1'] - loin_fs['y_test_1'])**2
loin_fs_test= loin_fs.pivot(index='type', columns='model', values='diff')
loin_fs_test= loin_fs_test.rename(rename_dict_regr, axis='columns')
loin_fs_test= loin_fs_test.rename(rename_dict_type)
loin_fs_test= loin_fs_test.loc[row_order, col_order]

loin_no_fs['diff']= (loin_no_fs['y_pred_1'] - loin_no_fs['y_test_1'])**2
loin_no_fs_test= loin_no_fs.pivot(index='type', columns='model', values='diff')
loin_no_fs_test= loin_no_fs_test.rename(rename_dict_regr, axis='columns')
loin_no_fs_test= loin_no_fs_test.rename(rename_dict_type)
loin_no_fs_test= loin_no_fs_test.loc[row_order, col_order]

In [20]:
loin_ttest_latex= test_compare(loin_fs_test, loin_no_fs_test).applymap(lambda x: '%.0e' % x).to_latex()
print(loin_ttest_latex)

#save results to tex file
with open(path_loin_ttest_latex, 'w') as f:
    f.write(latex_template.format(loin_ttest_latex))

\begin{tabular}{llllll}
\toprule
model & linear &    PLS &  lasso &  ridge &    kNN \\
type        &        &        &        &        &        \\
\midrule
multi-atlas &  3e-75 &  9e-52 &  4e-51 &  3e-63 &  1e-53 \\
mean        &  4e-45 &  7e-21 &  3e-21 &  5e-18 &  6e-35 \\
atlas 3     &  3e-35 &  1e-21 &  4e-25 &  2e-14 &  2e-39 \\
atlas 4     &  4e-52 &  1e-15 &  6e-24 &  8e-19 &  3e-28 \\
atlas 5     &  2e-32 &  2e-14 &  6e-14 &  1e-09 &  3e-17 \\
atlas 6     &  9e-67 &  6e-45 &  6e-35 &  1e-25 &  8e-31 \\
atlas 7     &  1e-60 &  2e-21 &  3e-12 &  3e-13 &  1e-41 \\
\bottomrule
\end{tabular}



In [21]:
loin_fs_ttest_diff_latex= test_diff(loin_fs_test).applymap(lambda x: '%.0e' % x).to_latex()
print(loin_fs_ttest_diff_latex)

#save results to tex file
with open(path_loin_fs_ttest_diff_latex, 'w') as f:
    f.write(latex_template.format(loin_fs_ttest_diff_latex))

\begin{tabular}{llllll}
\toprule
model & linear &    PLS &  lasso &  ridge &    kNN \\
type    &        &        &        &        &        \\
\midrule
mean    &  2e-03 &  3e-26 &  1e-16 &  1e-26 &  1e-11 \\
atlas 3 &  2e-07 &  3e-23 &  1e-18 &  1e-21 &  6e-14 \\
atlas 4 &  7e-01 &  1e-24 &  4e-11 &  3e-22 &  1e-16 \\
atlas 5 &  2e-10 &  3e-32 &  7e-32 &  5e-43 &  7e-10 \\
atlas 6 &  3e-05 &  3e-26 &  1e-23 &  3e-30 &  1e-25 \\
atlas 7 &  5e-03 &  1e-18 &  1e-16 &  1e-25 &  8e-16 \\
\bottomrule
\end{tabular}



In [22]:
loin_fs_ttest_diff_cross_latex= test_diff_cross(loin_fs_test.loc[['multi-atlas']]).applymap(lambda x: '%.0e' % x).to_latex()
print(loin_fs_ttest_diff_cross_latex)

#save results to tex file
with open(path_loin_fs_ttest_diff_cross_latex, 'w') as f:
    f.write(latex_template.format(loin_fs_ttest_diff_cross_latex))

\begin{tabular}{llllll}
\toprule
model & linear &    PLS &  lasso &  ridge &    kNN \\
model  &        &        &        &        &        \\
\midrule
linear &    nan &  2e-15 &  4e-10 &  2e-17 &  3e-14 \\
PLS    &  2e-15 &    nan &  2e-01 &  9e-03 &  5e-34 \\
lasso  &  4e-10 &  2e-01 &    nan &  1e-09 &  2e-27 \\
ridge  &  2e-17 &  9e-03 &  1e-09 &    nan &  3e-36 \\
kNN    &  3e-14 &  5e-34 &  2e-27 &  3e-36 &    nan \\
\bottomrule
\end{tabular}



## belly $r^2$ results

In [23]:
belly_fs_results= belly_fs.pivot(index='type', columns='model', values='r2_1')
belly_fs_results= belly_fs_results.rename(rename_dict_regr, axis='columns')
belly_fs_results= belly_fs_results.rename(rename_dict_type)
belly_fs_results= belly_fs_results.apply(lambda x: np.round(x, 4))
belly_fs_results= belly_fs_results.loc[row_order, col_order]

belly_no_fs_results= belly_no_fs.pivot(index='type', columns='model', values='r2_1')
belly_no_fs_results= belly_no_fs_results.rename(rename_dict_regr, axis='columns')
belly_no_fs_results= belly_no_fs_results.rename(rename_dict_type)
belly_no_fs_results= belly_no_fs_results.apply(lambda x: np.round(x, 4))
belly_no_fs_results= belly_no_fs_results.loc[row_order, col_order]

belly_all= pd.concat([belly_fs_results, belly_no_fs_results], axis='columns')
columns= pd.MultiIndex.from_product([['feature selection', 'no feature selection'], col_order])
belly_all.columns= columns

In [24]:
belly_all_r2_latex= belly_all.to_latex()
print(belly_all_r2_latex)

#save results to tex file
with open(path_belly_r2_latex, 'w') as f:
    f.write(latex_template.format(belly_all_r2_latex))

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} & \multicolumn{5}{l}{feature selection} & \multicolumn{5}{l}{no feature selection} \\
{} &            linear &     PLS &   lasso &   ridge &     kNN &               linear &     PLS &   lasso &   ridge &     kNN \\
type        &                   &         &         &         &         &                      &         &         &         &         \\
\midrule
multi-atlas &            0.9736 &  0.9786 &  0.9587 &  0.9964 &  0.9078 &               0.6653 &  0.8713 &  0.8687 &  0.8446 &  0.7679 \\
mean        &            0.9050 &  0.9056 &  0.9056 &  0.9058 &  0.8578 &               0.6368 &  0.8544 &  0.8824 &  0.8646 &  0.7264 \\
atlas 3     &            0.8613 &  0.8646 &  0.8728 &  0.8638 &  0.8361 &               0.6409 &  0.8103 &  0.8300 &  0.8096 &  0.6390 \\
atlas 4     &            0.8968 &  0.9016 &  0.9014 &  0.9036 &  0.8820 &               0.6059 &  0.8846 &  0.8847 &  0.8774 &  0.6774 \\
atlas 5     &            0.9286 &  0.9269 &  

## belly RMSE results

In [25]:
belly_fs_results_rmse= belly_fs.pivot(index='type', columns='model', values='rmse_1')
belly_fs_results_rmse= belly_fs_results_rmse.rename(rename_dict_regr, axis='columns')
belly_fs_results_rmse= belly_fs_results_rmse.rename(rename_dict_type)
belly_fs_results_rmse= belly_fs_results_rmse.apply(lambda x: np.round(x, 4))
belly_fs_results_rmse= belly_fs_results_rmse.loc[row_order, col_order]

belly_no_fs_results_rmse= belly_no_fs.pivot(index='type', columns='model', values='rmse_1')
belly_no_fs_results_rmse= belly_no_fs_results_rmse.rename(rename_dict_regr, axis='columns')
belly_no_fs_results_rmse= belly_no_fs_results_rmse.rename(rename_dict_type)
belly_no_fs_results_rmse= belly_no_fs_results_rmse.apply(lambda x: np.round(x, 4))
belly_no_fs_results_rmse= belly_no_fs_results_rmse.loc[row_order, col_order]

belly_all_rmse= pd.concat([belly_fs_results_rmse, belly_no_fs_results_rmse], axis='columns')
columns= pd.MultiIndex.from_product([['feature selection', 'no feature selection'], col_order])
belly_all_rmse.columns= columns

In [26]:
belly_all_rmse_latex= belly_all_rmse.to_latex()
print(belly_all_rmse_latex)

#save results to tex file
with open(path_belly_rmse_latex, 'w') as f:
    f.write(latex_template.format(belly_all_rmse_latex))

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} & \multicolumn{5}{l}{feature selection} & \multicolumn{5}{l}{no feature selection} \\
{} &            linear &       PLS &     lasso &     ridge &       kNN &               linear &       PLS &     lasso &     ridge &       kNN \\
type        &                   &           &           &           &           &                      &           &           &           &           \\
\midrule
multi-atlas &           87.7848 &   79.0851 &  109.8623 &   32.6244 &  164.1710 &             312.7792 &  193.9450 &  195.8674 &  213.1023 &  260.4372 \\
mean        &          166.6425 &  166.1032 &  166.1156 &  165.9333 &  203.8315 &             325.8013 &  206.2560 &  185.4082 &  198.9191 &  282.7861 \\
atlas 3     &          201.3335 &  198.9635 &  192.8346 &  199.5297 &  218.8997 &             323.9648 &  235.4361 &  222.9344 &  235.9226 &  324.8045 \\
atlas 4     &          173.6765 &  169.5659 &  169.7415 &  167.8342 &  185.6749 &             339.3978 

## t-test belly

In [27]:
belly_fs['diff']= (belly_fs['y_pred_1'] - belly_fs['y_test_1'])**2
belly_fs_test= belly_fs.pivot(index='type', columns='model', values='diff')
belly_fs_test= belly_fs_test.rename(rename_dict_regr, axis='columns')
belly_fs_test= belly_fs_test.rename(rename_dict_type)
belly_fs_test= belly_fs_test.loc[row_order, col_order]

belly_no_fs['diff']= (belly_no_fs['y_pred_1'] - belly_no_fs['y_test_1'])**2
belly_no_fs_test= belly_no_fs.pivot(index='type', columns='model', values='diff')
belly_no_fs_test= belly_no_fs_test.rename(rename_dict_regr, axis='columns')
belly_no_fs_test= belly_no_fs_test.rename(rename_dict_type)
belly_no_fs_test= belly_no_fs_test.loc[row_order, col_order]

In [28]:
belly_ttest_latex= test_compare(belly_fs_test, belly_no_fs_test).applymap(lambda x: '%.0e' % x).to_latex()
print(belly_ttest_latex)

#save results to tex file
with open(path_belly_ttest_latex, 'w') as f:
    f.write(latex_template.format(belly_ttest_latex))

\begin{tabular}{llllll}
\toprule
model & linear &    PLS &  lasso &  ridge &    kNN \\
type        &        &        &        &        &        \\
\midrule
multi-atlas &  1e-87 &  3e-42 &  8e-43 &  1e-78 &  2e-54 \\
mean        &  6e-53 &  1e-18 &  3e-18 &  2e-21 &  6e-38 \\
atlas 3     &  7e-17 &  4e-17 &  1e-23 &  7e-29 &  7e-53 \\
atlas 4     &  5e-37 &  2e-06 &  2e-18 &  7e-26 &  1e-52 \\
atlas 5     &  3e-32 &  9e-35 &  8e-09 &  3e-28 &  2e-38 \\
atlas 6     &  2e-32 &  3e-26 &  2e-15 &  1e-32 &  2e-28 \\
atlas 7     &  2e-46 &  2e-16 &  6e-27 &  2e-23 &  3e-19 \\
\bottomrule
\end{tabular}



In [29]:
belly_fs_ttest_diff_latex= test_diff(belly_fs_test).applymap(lambda x: '%.0e' % x).to_latex()
print(belly_fs_ttest_diff_latex)

#save results to tex file
with open(path_belly_fs_ttest_diff_latex, 'w') as f:
    f.write(latex_template.format(belly_fs_ttest_diff_latex))

\begin{tabular}{llllll}
\toprule
model & linear &    PLS &  lasso &  ridge &    kNN \\
type    &        &        &        &        &        \\
\midrule
mean    &  4e-39 &  3e-42 &  8e-28 &  9e-63 &  6e-14 \\
atlas 3 &  2e-50 &  5e-50 &  4e-39 &  3e-70 &  1e-16 \\
atlas 4 &  3e-35 &  9e-42 &  1e-23 &  6e-60 &  2e-05 \\
atlas 5 &  5e-32 &  2e-35 &  2e-32 &  4e-62 &  1e-03 \\
atlas 6 &  3e-32 &  1e-32 &  8e-23 &  2e-54 &  5e-19 \\
atlas 7 &  2e-57 &  3e-36 &  2e-36 &  3e-63 &  4e-11 \\
\bottomrule
\end{tabular}



In [30]:
belly_fs_ttest_diff_cross_latex= test_diff_cross(belly_fs_test.loc[['multi-atlas']]).applymap(lambda x: '%.0e' % x).to_latex()
print(belly_fs_ttest_diff_cross_latex)

#save results to tex file
with open(path_belly_fs_ttest_diff_cross_latex, 'w') as f:
    f.write(latex_template.format(belly_fs_ttest_diff_cross_latex))

\begin{tabular}{llllll}
\toprule
model & linear &    PLS &  lasso &  ridge &    kNN \\
model  &        &        &        &        &        \\
\midrule
linear &    nan &  3e-04 &  2e-16 &  2e-69 &  6e-44 \\
PLS    &  3e-04 &    nan &  7e-33 &  1e-65 &  6e-48 \\
lasso  &  2e-16 &  7e-33 &    nan &  5e-88 &  9e-26 \\
ridge  &  2e-69 &  1e-65 &  5e-88 &    nan &  2e-73 \\
kNN    &  6e-44 &  6e-48 &  9e-26 &  2e-73 &    nan \\
\bottomrule
\end{tabular}



## groin $r^2$ results

In [31]:
groin_fs_results= groin_fs.pivot(index='type', columns='model', values='r2_1')
groin_fs_results= groin_fs_results.rename(rename_dict_regr, axis='columns')
groin_fs_results= groin_fs_results.rename(rename_dict_type)
groin_fs_results= groin_fs_results.apply(lambda x: np.round(x, 4))
groin_fs_results= groin_fs_results.loc[row_order, col_order]

groin_no_fs_results= groin_no_fs.pivot(index='type', columns='model', values='r2_1')
groin_no_fs_results= groin_no_fs_results.rename(rename_dict_regr, axis='columns')
groin_no_fs_results= groin_no_fs_results.rename(rename_dict_type)
groin_no_fs_results= groin_no_fs_results.apply(lambda x: np.round(x, 4))
groin_no_fs_results= groin_no_fs_results.loc[row_order, col_order]

groin_all= pd.concat([groin_fs_results, groin_no_fs_results], axis='columns')
columns= pd.MultiIndex.from_product([['feature selection', 'no feature selection'], col_order])
groin_all.columns= columns

In [32]:
groin_all_r2_latex= groin_all.to_latex()
print(groin_all_r2_latex)

#save results to tex file
with open(path_groin_r2_latex, 'w') as f:
    f.write(latex_template.format(groin_all_r2_latex))

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} & \multicolumn{5}{l}{feature selection} & \multicolumn{5}{l}{no feature selection} \\
{} &            linear &     PLS &   lasso &   ridge &     kNN &               linear &     PLS &   lasso &   ridge &     kNN \\
type        &                   &         &         &         &         &                      &         &         &         &         \\
\midrule
multi-atlas &            0.8616 &  0.7404 &  0.9059 &  0.9543 &  0.6981 &               0.3275 &  0.4847 &  0.5240 &  0.4554 &  0.4991 \\
mean        &            0.7156 &  0.7097 &  0.7026 &  0.7042 &  0.6491 &              -0.1917 &  0.5097 &  0.5576 &  0.5402 &  0.3786 \\
atlas 3     &            0.6024 &  0.5750 &  0.6559 &  0.6002 &  0.5197 &              -0.2565 &  0.4390 &  0.4704 &  0.5020 &  0.3882 \\
atlas 4     &            0.6266 &  0.7428 &  0.7437 &  0.7754 &  0.5706 &               0.0498 &  0.5592 &  0.6699 &  0.5972 &  0.4934 \\
atlas 5     &            0.6827 &  0.6083 &  

## groin RMSE results

In [33]:
groin_fs_results_rmse= groin_fs.pivot(index='type', columns='model', values='rmse_1')
groin_fs_results_rmse= groin_fs_results_rmse.rename(rename_dict_regr, axis='columns')
groin_fs_results_rmse= groin_fs_results_rmse.rename(rename_dict_type)
groin_fs_results_rmse= groin_fs_results_rmse.apply(lambda x: np.round(x, 4))
groin_fs_results_rmse= groin_fs_results_rmse.loc[row_order, col_order]

groin_no_fs_results_rmse= groin_no_fs.pivot(index='type', columns='model', values='rmse_1')
groin_no_fs_results_rmse= groin_no_fs_results_rmse.rename(rename_dict_regr, axis='columns')
groin_no_fs_results_rmse= groin_no_fs_results_rmse.rename(rename_dict_type)
groin_no_fs_results_rmse= groin_no_fs_results_rmse.apply(lambda x: np.round(x, 4))
groin_no_fs_results_rmse= groin_no_fs_results_rmse.loc[row_order, col_order]

groin_all_rmse= pd.concat([groin_fs_results_rmse, groin_no_fs_results_rmse], axis='columns')
columns= pd.MultiIndex.from_product([['feature selection', 'no feature selection'], col_order])
groin_all_rmse.columns= columns

In [34]:
groin_all_rmse_latex= groin_all_rmse.to_latex()
print(groin_all_rmse_latex)

#save results to tex file
with open(path_groin_rmse_latex, 'w') as f:
    f.write(latex_template.format(groin_all_rmse_latex))

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} & \multicolumn{5}{l}{feature selection} & \multicolumn{5}{l}{no feature selection} \\
{} &            linear &       PLS &     lasso &     ridge &       kNN &               linear &       PLS &     lasso &     ridge &       kNN \\
type        &                   &           &           &           &           &                      &           &           &           &           \\
\midrule
multi-atlas &           81.0252 &  110.9624 &   66.8018 &   46.5339 &  119.6655 &             178.5928 &  156.3376 &  150.2591 &  160.7118 &  154.1377 \\
mean        &          116.1414 &  117.3363 &  118.7637 &  118.4438 &  129.0095 &             237.7417 &  152.4944 &  144.8595 &  147.6803 &  171.6745 \\
atlas 3     &          137.3250 &  141.9778 &  127.7560 &  137.6931 &  150.9351 &             244.1151 &  163.1191 &  158.4790 &  153.6901 &  170.3478 \\
atlas 4     &          133.0712 &  110.4467 &  110.2605 &  103.2062 &  142.7123 &             212.2869 

## t-test groin

In [35]:
groin_fs['diff']= (groin_fs['y_pred_1'] - groin_fs['y_test_1'])**2
groin_fs_test= groin_fs.pivot(index='type', columns='model', values='diff')
groin_fs_test= groin_fs_test.rename(rename_dict_regr, axis='columns')
groin_fs_test= groin_fs_test.rename(rename_dict_type)
groin_fs_test= groin_fs_test.loc[row_order, col_order]

groin_no_fs['diff']= (groin_no_fs['y_pred_1'] - groin_no_fs['y_test_1'])**2
groin_no_fs_test= groin_no_fs.pivot(index='type', columns='model', values='diff')
groin_no_fs_test= groin_no_fs_test.rename(rename_dict_regr, axis='columns')
groin_no_fs_test= groin_no_fs_test.rename(rename_dict_type)
groin_no_fs_test= groin_no_fs_test.loc[row_order, col_order]

In [36]:
groin_ttest_latex= test_compare(groin_fs_test, groin_no_fs_test).applymap(lambda x: '%.0e' % x).to_latex()
print(groin_ttest_latex)

#save results to tex file
with open(path_groin_ttest_latex, 'w') as f:
    f.write(latex_template.format(groin_ttest_latex))

\begin{tabular}{llllll}
\toprule
model & linear &    PLS &  lasso &  ridge &    kNN \\
type        &        &        &        &        &        \\
\midrule
multi-atlas &  2e-21 &  2e-43 &  4e-58 &  3e-43 &  3e-22 \\
mean        &  3e-30 &  2e-21 &  3e-17 &  2e-21 &  6e-20 \\
atlas 3     &  2e-39 &  1e-31 &  1e-15 &  6e-08 &  2e-10 \\
atlas 4     &  1e-04 &  3e-22 &  1e-11 &  2e-39 &  1e-06 \\
atlas 5     &  5e-31 &  3e-22 &  2e-23 &  4e-26 &  7e-11 \\
atlas 6     &  6e-18 &  5e-14 &  6e-03 &  8e-08 &  4e-14 \\
atlas 7     &  7e-08 &  2e-18 &  6e-23 &  6e-16 &  4e-20 \\
\bottomrule
\end{tabular}



In [37]:
groin_fs_ttest_diff_latex= test_diff(groin_fs_test).applymap(lambda x: '%.0e' % x).to_latex()
print(groin_fs_ttest_diff_latex)

#save results to tex file
with open(path_groin_fs_ttest_diff_latex, 'w') as f:
    f.write(latex_template.format(groin_fs_ttest_diff_latex))

\begin{tabular}{llllll}
\toprule
model & linear &    PLS &  lasso &   ridge &    kNN \\
type    &        &        &        &         &        \\
\midrule
mean    &  3e-36 &  1e-02 &  4e-39 &   6e-70 &  2e-06 \\
atlas 3 &  9e-50 &  4e-18 &  1e-38 &   2e-72 &  3e-13 \\
atlas 4 &  5e-34 &  8e-01 &  8e-54 &   2e-59 &  1e-17 \\
atlas 5 &  2e-31 &  2e-20 &  6e-46 &   1e-61 &  7e-31 \\
atlas 6 &  2e-25 &  9e-16 &  6e-37 &   1e-81 &  4e-03 \\
atlas 7 &  6e-32 &  3e-03 &  2e-46 &  5e-110 &  5e-15 \\
\bottomrule
\end{tabular}



In [38]:
groin_fs_ttest_diff_cross_latex= test_diff_cross(groin_fs_test.loc[['multi-atlas']]).applymap(lambda x: '%.0e' % x).to_latex()
print(groin_fs_ttest_diff_cross_latex)

#save results to tex file
with open(path_groin_fs_ttest_diff_cross_latex, 'w') as f:
    f.write(latex_template.format(groin_fs_ttest_diff_cross_latex))

\begin{tabular}{llllll}
\toprule
model & linear &    PLS &  lasso &  ridge &    kNN \\
model  &        &        &        &        &        \\
\midrule
linear &    nan &  1e-24 &  1e-10 &  3e-27 &  4e-23 \\
PLS    &  1e-24 &    nan &  5e-45 &  7e-59 &  5e-03 \\
lasso  &  1e-10 &  5e-45 &    nan &  5e-20 &  8e-35 \\
ridge  &  3e-27 &  7e-59 &  5e-20 &    nan &  8e-43 \\
kNN    &  4e-23 &  5e-03 &  8e-35 &  8e-43 &    nan \\
\bottomrule
\end{tabular}



## shoulder $r^2$ results

In [39]:
shoulder_fs_results= shoulder_fs.pivot(index='type', columns='model', values='r2_1')
shoulder_fs_results= shoulder_fs_results.rename(rename_dict_regr, axis='columns')
shoulder_fs_results= shoulder_fs_results.rename(rename_dict_type)
shoulder_fs_results= shoulder_fs_results.apply(lambda x: np.round(x, 4))
shoulder_fs_results= shoulder_fs_results.loc[row_order, col_order]

shoulder_no_fs_results= shoulder_no_fs.pivot(index='type', columns='model', values='r2_1')
shoulder_no_fs_results= shoulder_no_fs_results.rename(rename_dict_regr, axis='columns')
shoulder_no_fs_results= shoulder_no_fs_results.rename(rename_dict_type)
shoulder_no_fs_results= shoulder_no_fs_results.apply(lambda x: np.round(x, 4))
shoulder_no_fs_results= shoulder_no_fs_results.loc[row_order, col_order]

shoulder_all= pd.concat([shoulder_fs_results, shoulder_no_fs_results], axis='columns')
columns= pd.MultiIndex.from_product([['feature selection', 'no feature selection'], col_order])
shoulder_all.columns= columns

In [40]:
shoulder_all_r2_latex= shoulder_all.to_latex()
print(shoulder_all_r2_latex)

#save results to tex file
with open(path_shoulder_r2_latex, 'w') as f:
    f.write(latex_template.format(shoulder_all_r2_latex))

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} & \multicolumn{5}{l}{feature selection} & \multicolumn{5}{l}{no feature selection} \\
{} &            linear &     PLS &   lasso &   ridge &     kNN &               linear &     PLS &   lasso &   ridge &     kNN \\
type        &                   &         &         &         &         &                      &         &         &         &         \\
\midrule
multi-atlas &            0.9962 &  0.9531 &  0.9909 &  0.9893 &  0.8598 &               0.7753 &  0.8896 &  0.8962 &  0.8896 &  0.6688 \\
mean        &            0.8972 &  0.9177 &  0.8971 &  0.9012 &  0.8272 &               0.6895 &  0.8795 &  0.8825 &  0.8843 &  0.6894 \\
atlas 3     &            0.9181 &  0.9176 &  0.9182 &  0.9193 &  0.8289 &               0.7890 &  0.8857 &  0.8997 &  0.8948 &  0.6985 \\
atlas 4     &            0.9358 &  0.9083 &  0.9272 &  0.9274 &  0.8074 &               0.7351 &  0.8813 &  0.8820 &  0.8955 &  0.6246 \\
atlas 5     &            0.9194 &  0.9133 &  

## shoulder RMSE results

In [41]:
shoulder_fs_results_rmse= shoulder_fs.pivot(index='type', columns='model', values='rmse_1')
shoulder_fs_results_rmse= shoulder_fs_results_rmse.rename(rename_dict_regr, axis='columns')
shoulder_fs_results_rmse= shoulder_fs_results_rmse.rename(rename_dict_type)
shoulder_fs_results_rmse= shoulder_fs_results_rmse.apply(lambda x: np.round(x, 4))
shoulder_fs_results_rmse= shoulder_fs_results_rmse.loc[row_order, col_order]

shoulder_no_fs_results_rmse= shoulder_no_fs.pivot(index='type', columns='model', values='rmse_1')
shoulder_no_fs_results_rmse= shoulder_no_fs_results_rmse.rename(rename_dict_regr, axis='columns')
shoulder_no_fs_results_rmse= shoulder_no_fs_results_rmse.rename(rename_dict_type)
shoulder_no_fs_results_rmse= shoulder_no_fs_results_rmse.apply(lambda x: np.round(x, 4))
shoulder_no_fs_results_rmse= shoulder_no_fs_results_rmse.loc[row_order, col_order]

shoulder_all_rmse= pd.concat([shoulder_fs_results_rmse, shoulder_no_fs_results_rmse], axis='columns')
columns= pd.MultiIndex.from_product([['feature selection', 'no feature selection'], col_order])
shoulder_all_rmse.columns= columns

In [42]:
shoulder_all_rmse_latex= shoulder_all_rmse.to_latex()
print(shoulder_all_rmse_latex)

#save results to tex file
with open(path_shoulder_rmse_latex, 'w') as f:
    f.write(latex_template.format(shoulder_all_rmse_latex))

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} & \multicolumn{5}{l}{feature selection} & \multicolumn{5}{l}{no feature selection} \\
{} &            linear &       PLS &     lasso &     ridge &       kNN &               linear &       PLS &     lasso &     ridge &       kNN \\
type        &                   &           &           &           &           &                      &           &           &           &           \\
\midrule
multi-atlas &           40.8138 &  143.2308 &   63.0590 &   68.3362 &  247.7341 &             313.6371 &  219.9051 &  213.2199 &  219.8397 &  380.8014 \\
mean        &          212.1423 &  189.7871 &  212.2223 &  207.9466 &  275.0877 &             368.7072 &  229.7213 &  226.8033 &  225.0904 &  368.8012 \\
atlas 3     &          189.3588 &  189.9674 &  189.1992 &  187.9753 &  273.7420 &             303.9221 &  223.6897 &  209.5963 &  214.5776 &  363.3477 \\
atlas 4     &          167.6654 &  200.3755 &  178.5645 &  178.2766 &  290.4043 &             340.5695 

## t-test shoulder

In [43]:
shoulder_fs['diff']= (shoulder_fs['y_pred_1'] - shoulder_fs['y_test_1'])**2
shoulder_fs_test= shoulder_fs.pivot(index='type', columns='model', values='diff')
shoulder_fs_test= shoulder_fs_test.rename(rename_dict_regr, axis='columns')
shoulder_fs_test= shoulder_fs_test.rename(rename_dict_type)
shoulder_fs_test= shoulder_fs_test.loc[row_order, col_order]

shoulder_no_fs['diff']= (shoulder_no_fs['y_pred_1'] - shoulder_no_fs['y_test_1'])**2
shoulder_no_fs_test= shoulder_no_fs.pivot(index='type', columns='model', values='diff')
shoulder_no_fs_test= shoulder_no_fs_test.rename(rename_dict_regr, axis='columns')
shoulder_no_fs_test= shoulder_no_fs_test.rename(rename_dict_type)
shoulder_no_fs_test= shoulder_no_fs_test.loc[row_order, col_order]

In [44]:
shoulder_ttest_latex= test_compare(shoulder_fs_test, shoulder_no_fs_test).applymap(lambda x: '%.0e' % x).to_latex()
print(shoulder_ttest_latex)

#save results to tex file
with open(path_shoulder_ttest_latex, 'w') as f:
    f.write(latex_template.format(shoulder_ttest_latex))

\begin{tabular}{llllll}
\toprule
model & linear &    PLS &  lasso &  ridge &    kNN \\
type        &        &        &        &        &        \\
\midrule
multi-atlas &  4e-94 &  2e-78 &  1e-72 &  9e-80 &  8e-33 \\
mean        &  4e-24 &  4e-39 &  3e-08 &  1e-08 &  7e-21 \\
atlas 3     &  3e-32 &  1e-17 &  2e-18 &  6e-19 &  2e-37 \\
atlas 4     &  8e-47 &  2e-12 &  2e-23 &  8e-25 &  3e-34 \\
atlas 5     &  2e-39 &  2e-39 &  1e-24 &  4e-36 &  2e-61 \\
atlas 6     &  7e-18 &  1e-37 &  2e-19 &  2e-18 &  1e-25 \\
atlas 7     &  3e-55 &  2e-43 &  6e-42 &  4e-38 &  3e-23 \\
\bottomrule
\end{tabular}



In [45]:
shoulder_fs_ttest_diff_latex= test_diff(shoulder_fs_test).applymap(lambda x: '%.0e' % x).to_latex()
print(shoulder_fs_ttest_diff_latex)

#save results to tex file
with open(path_shoulder_fs_ttest_diff_latex, 'w') as f:
    f.write(latex_template.format(shoulder_fs_ttest_diff_latex))

\begin{tabular}{llllll}
\toprule
model & linear &    PLS &  lasso &  ridge &    kNN \\
type    &        &        &        &        &        \\
\midrule
mean    &  3e-71 &  1e-25 &  3e-65 &  5e-68 &  1e-05 \\
atlas 3 &  1e-86 &  1e-26 &  8e-69 &  5e-65 &  4e-04 \\
atlas 4 &  3e-88 &  5e-36 &  7e-99 &  2e-95 &  1e-09 \\
atlas 5 &  7e-81 &  4e-37 &  3e-71 &  2e-70 &  3e-14 \\
atlas 6 &  1e-76 &  3e-22 &  6e-70 &  1e-76 &  3e-21 \\
atlas 7 &  4e-95 &  3e-39 &  3e-85 &  5e-85 &  2e-28 \\
\bottomrule
\end{tabular}



In [46]:
shoulder_fs_ttest_diff_cross_latex= test_diff_cross(shoulder_fs_test.loc[['multi-atlas']]).applymap(lambda x: '%.0e' % x).to_latex()
print(shoulder_fs_ttest_diff_cross_latex)

#save results to tex file
with open(path_shoulder_fs_ttest_diff_cross_latex, 'w') as f:
    f.write(latex_template.format(shoulder_fs_ttest_diff_cross_latex))

\begin{tabular}{llllll}
\toprule
model & linear &    PLS &  lasso &  ridge &    kNN \\
model  &        &        &        &        &        \\
\midrule
linear &    nan &  1e-88 &  8e-34 &  1e-41 &  2e-38 \\
PLS    &  1e-88 &    nan &  2e-72 &  2e-68 &  7e-20 \\
lasso  &  8e-34 &  2e-72 &    nan &  2e-03 &  4e-36 \\
ridge  &  1e-41 &  2e-68 &  2e-03 &    nan &  1e-35 \\
kNN    &  2e-38 &  7e-20 &  4e-36 &  1e-35 &    nan \\
\bottomrule
\end{tabular}



## body $r^2$ results

In [47]:
body_fs_results= body_fs.pivot(index='type', columns='model', values='r2_1')
body_fs_results= body_fs_results.rename(rename_dict_regr, axis='columns')
body_fs_results= body_fs_results.rename(rename_dict_type)
body_fs_results= body_fs_results.apply(lambda x: np.round(x, 4))
body_fs_results= body_fs_results.loc[row_order, col_order]

body_no_fs_results= body_no_fs.pivot(index='type', columns='model', values='r2_1')
body_no_fs_results= body_no_fs_results.rename(rename_dict_regr, axis='columns')
body_no_fs_results= body_no_fs_results.rename(rename_dict_type)
body_no_fs_results= body_no_fs_results.apply(lambda x: np.round(x, 4))
body_no_fs_results= body_no_fs_results.loc[row_order, col_order]

body_all= pd.concat([body_fs_results, body_no_fs_results], axis='columns')
columns= pd.MultiIndex.from_product([['feature selection', 'no feature selection'], col_order])
body_all.columns= columns

In [48]:
body_all_r2_latex= body_all.to_latex()
print(body_all_r2_latex)

#save results to tex file
with open(path_body_r2_latex, 'w') as f:
    f.write(latex_template.format(body_all_r2_latex))

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} & \multicolumn{5}{l}{feature selection} & \multicolumn{5}{l}{no feature selection} \\
{} &            linear &     PLS &   lasso &   ridge &     kNN &               linear &     PLS &   lasso &   ridge &     kNN \\
type        &                   &         &         &         &         &                      &         &         &         &         \\
\midrule
multi-atlas &            0.9885 &  0.9894 &  0.9891 &  0.9895 &  0.9129 &               0.8898 &  0.9739 &  0.9704 &  0.9724 &  0.5715 \\
mean        &            0.9842 &  0.9849 &  0.9843 &  0.9853 &  0.9031 &               0.4056 &  0.9727 &  0.9753 &  0.9734 &  0.6263 \\
atlas 3     &            0.9848 &  0.9818 &  0.9847 &  0.9853 &  0.8765 &              -0.4333 &  0.9674 &  0.9664 &  0.9647 &  0.6348 \\
atlas 4     &            0.9824 &  0.9868 &  0.9826 &  0.9868 &  0.8712 &              -2.5919 &  0.9702 &  0.9691 &  0.9656 &  0.5600 \\
atlas 5     &            0.9845 &  0.9843 &  

## body RMSE results

In [49]:
body_fs_results_rmse= body_fs.pivot(index='type', columns='model', values='rmse_1')
body_fs_results_rmse= body_fs_results_rmse.rename(rename_dict_regr, axis='columns')
body_fs_results_rmse= body_fs_results_rmse.rename(rename_dict_type)
body_fs_results_rmse= body_fs_results_rmse.apply(lambda x: np.round(x, 4))
body_fs_results_rmse= body_fs_results_rmse.loc[row_order, col_order]

body_no_fs_results_rmse= body_no_fs.pivot(index='type', columns='model', values='rmse_1')
body_no_fs_results_rmse= body_no_fs_results_rmse.rename(rename_dict_regr, axis='columns')
body_no_fs_results_rmse= body_no_fs_results_rmse.rename(rename_dict_type)
body_no_fs_results_rmse= body_no_fs_results_rmse.apply(lambda x: np.round(x, 4))
body_no_fs_results_rmse= body_no_fs_results_rmse.loc[row_order, col_order]

body_all_rmse= pd.concat([body_fs_results_rmse, body_no_fs_results_rmse], axis='columns')
columns= pd.MultiIndex.from_product([['feature selection', 'no feature selection'], col_order])
body_all_rmse.columns= columns

In [50]:
body_all_rmse_latex= body_all_rmse.to_latex()
print(body_all_rmse_latex)

#save results to tex file
with open(path_body_rmse_latex, 'w') as f:
    f.write(latex_template.format(body_all_rmse_latex))

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} & \multicolumn{5}{l}{feature selection} & \multicolumn{5}{l}{no feature selection} \\
{} &            linear &       PLS &     lasso &     ridge &        kNN &               linear &       PLS &     lasso &     ridge &        kNN \\
type        &                   &           &           &           &            &                      &           &           &           &            \\
\midrule
multi-atlas &          524.1614 &  503.2498 &  511.6127 &  501.8615 &  1444.9752 &            1624.9085 &  790.5371 &  842.8359 &  812.6658 &  3204.5864 \\
mean        &          615.4048 &  602.1268 &  612.7417 &  593.8281 &  1523.5512 &            3773.9431 &  808.5480 &  768.6579 &  798.7240 &  2992.3469 \\
atlas 3     &          603.9188 &  659.6006 &  604.7777 &  593.9343 &  1720.6046 &            5860.6153 &  883.7403 &  897.8557 &  920.3191 &  2958.3145 \\
atlas 4     &          649.9304 &  562.3052 &  645.4266 &  563.1414 &  1757.1360 &           

## t-test body

In [51]:
body_fs['diff']= (body_fs['y_pred_1'] - body_fs['y_test_1'])**2
body_fs_test= body_fs.pivot(index='type', columns='model', values='diff')
body_fs_test= body_fs_test.rename(rename_dict_regr, axis='columns')
body_fs_test= body_fs_test.rename(rename_dict_type)
body_fs_test= body_fs_test.loc[row_order, col_order]

body_no_fs['diff']= (body_no_fs['y_pred_1'] - body_no_fs['y_test_1'])**2
body_no_fs_test= body_no_fs.pivot(index='type', columns='model', values='diff')
body_no_fs_test= body_no_fs_test.rename(rename_dict_regr, axis='columns')
body_no_fs_test= body_no_fs_test.rename(rename_dict_type)
body_no_fs_test= body_no_fs_test.loc[row_order, col_order]

In [52]:
body_ttest_latex= test_compare(body_fs_test, body_no_fs_test).applymap(lambda x: '%.0e' % x).to_latex()
print(body_ttest_latex)

#save results to tex file
with open(path_body_ttest_latex, 'w') as f:
    f.write(latex_template.format(body_ttest_latex))

\begin{tabular}{llllll}
\toprule
model & linear &    PLS &  lasso &  ridge &    kNN \\
type        &        &        &        &        &        \\
\midrule
multi-atlas &  6e-76 &  1e-62 &  1e-55 &  2e-45 &  1e-32 \\
mean        &  3e-30 &  5e-28 &  5e-38 &  6e-31 &  1e-29 \\
atlas 3     &  2e-21 &  2e-53 &  2e-37 &  3e-66 &  2e-28 \\
atlas 4     &  4e-09 &  1e-45 &  9e-48 &  3e-55 &  6e-34 \\
atlas 5     &  1e-03 &  1e-30 &  2e-70 &  7e-62 &  1e-35 \\
atlas 6     &  6e-08 &  6e-15 &  4e-44 &  1e-23 &  5e-32 \\
atlas 7     &  3e-15 &  2e-30 &  1e-46 &  6e-31 &  1e-34 \\
\bottomrule
\end{tabular}



In [53]:
body_fs_ttest_diff_latex= test_diff(body_fs_test).applymap(lambda x: '%.0e' % x).to_latex()
print(body_fs_ttest_diff_latex)

#save results to tex file
with open(path_body_fs_ttest_diff_latex, 'w') as f:
    f.write(latex_template.format(body_fs_ttest_diff_latex))

\begin{tabular}{llllll}
\toprule
model & linear &    PLS &  lasso &  ridge &    kNN \\
type    &        &        &        &        &        \\
\midrule
mean    &  3e-09 &  6e-26 &  2e-21 &  1e-11 &  7e-03 \\
atlas 3 &  4e-14 &  5e-36 &  3e-15 &  4e-15 &  3e-11 \\
atlas 4 &  5e-17 &  5e-07 &  2e-23 &  2e-08 &  1e-16 \\
atlas 5 &  5e-13 &  3e-34 &  3e-17 &  2e-06 &  1e-04 \\
atlas 6 &  1e-03 &  3e-13 &  9e-12 &  2e-07 &  1e-29 \\
atlas 7 &  9e-26 &  1e-39 &  4e-47 &  6e-39 &  3e-17 \\
\bottomrule
\end{tabular}



In [54]:
body_fs_ttest_diff_cross_latex= test_diff_cross(body_fs_test.loc[['multi-atlas']]).applymap(lambda x: '%.0e' % x).to_latex()
print(body_fs_ttest_diff_cross_latex)

#save results to tex file
with open(path_body_fs_ttest_diff_cross_latex, 'w') as f:
    f.write(latex_template.format(body_fs_ttest_diff_cross_latex))

\begin{tabular}{llllll}
\toprule
model & linear &    PLS &  lasso &  ridge &    kNN \\
model  &        &        &        &        &        \\
\midrule
linear &    nan &  2e-02 &  2e-01 &  2e-02 &  6e-42 \\
PLS    &  2e-02 &    nan &  3e-01 &  9e-01 &  6e-42 \\
lasso  &  2e-01 &  3e-01 &    nan &  3e-01 &  5e-42 \\
ridge  &  2e-02 &  9e-01 &  3e-01 &    nan &  3e-42 \\
kNN    &  6e-42 &  6e-42 &  5e-42 &  3e-42 &    nan \\
\bottomrule
\end{tabular}



### Convert tex files to pdfs

In [55]:
import shutil
import glob

pdflatex_path= shutil.which("pdflatex")
if path_prefix_results != os.path.basename(os.getcwd()):
    os.chdir(path_prefix_results)
texs= glob.glob("*tex")
temp = "tmp"
os.makedirs(temp, exist_ok=True)

In [56]:
for tex in texs:
    print(f"Converting {tex}...")
    cmd= f"{pdflatex_path} {tex} >> /dev/null"
    os.system(cmd)
    for i in glob.glob("*.aux") + glob.glob("*.log"):
        shutil.move(os.path.abspath(i), os.path.join(os.path.abspath(temp),i))

Converting belly_fs_ttest_diff_cross.tex...
Converting loin_r2.tex...
Converting leg_fs_ttest_diff_cross.tex...
Converting belly_fs_ttest_diff.tex...
Converting groin_fs_ttest_diff_cross.tex...
Converting leg_fs_ttest_diff.tex...
Converting groin_r2.tex...
Converting shoulder_fs_ttest_diff.tex...
Converting shoulder_fs_ttest_diff_cross.tex...
Converting body_ttest.tex...
Converting groin_fs_ttest_diff.tex...
Converting body_fs_ttest_diff.tex...
Converting groin_rmse.tex...
Converting body_rmse.tex...
Converting belly_ttest.tex...
Converting loin_rmse.tex...
Converting loin_fs_ttest_diff_cross.tex...
Converting shoulder_r2.tex...
Converting shoulder_ttest.tex...
Converting leg_rmse.tex...
Converting leg_ttest.tex...
Converting shoulder_rmse.tex...
Converting leg_r2.tex...
Converting body_fs_ttest_diff_cross.tex...
Converting loin_fs_ttest_diff.tex...
Converting belly_rmse.tex...
Converting loin_ttest.tex...
Converting groin_ttest.tex...
Converting body_r2.tex...
Converting belly_r2.tex.